In [2]:
import pandas as pd
import gzip

In [3]:
games = pd.read_csv(r'data_sets\steam_gamez.csv',index_col=False)

C:\Users\jonat\AppData\Local\Temp\ipykernel_8632\2807454174.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  games = pd.read_csv(r'data_sets\steam_gamez.csv',index_col=False)


In [4]:
# Verifico que las filas que tengo pensado eliminar, solo esten rellenas con valores NaN, de lo contrario, en pasos siguientes verificare como realizar la normalizacion
games[games.isna().all(axis=1)].shape[0] + games.dropna(thresh=1).shape[0]

120445

In [5]:
# Elimino las filas nulas


games.dropna(thresh=9,inplace=True) # ---> Elimino las filas que cumplen con la condicion de estar completamente sin datos (NaN), de lo contrario si contienen algun valor no se eliminan


games.drop(columns='title',inplace=True) # ---> Elimino la columna title ya que la columna app_name contien el mismo valor y con los faltantes


games = games.drop(games.loc[games['id'].isna()].index) # ---> Realice una exploracion en la columna id, y tome la desicion de eliminar las filas que no contienen valores, no tienen relevancia en el resto del proceso


games['id'] = games['id'].astype(int) # ---> Realice una conversion en el tipo de dato de la columna id, de Float a Int


games = games.drop_duplicates(subset=['id']) # ---> Comprobe que existen valores repetidos, verifique que todos sus valores sean identicos y luego procedi a realizar su eliminacion.


# Primero comprobe que la columna genres y tags, contienen los mismos valores, complete los faltantes de la columna tags con los valores de la columna genres
filas_vacias = games['tags'].isna() # ---> guardo las filas con valores NaN en tags
games.loc[filas_vacias, 'tags'] = games.loc[filas_vacias, 'genres'] # asigno los valores de la columna genres dentro de la columna tags
games.drop(columns='genres',inplace=True) # ---> Elimino la columna genres


# Relleno los valores de la columna publisher, con los valores de la columna developer, solo cuando publisher no tiene ningun valor.
filas_vacias = games['publisher'].isna()
games.loc[filas_vacias, 'publisher'] = games.loc[filas_vacias, 'developer']
# En este caso no elimino ninguna de las columnas, ya que algunos valores que son relevantes tambien son diferentes.

games['price'] = games['price'].replace('Free', 'Free to Play')

games.dropna(inplace=True) # ---> Elimino todos los NaN

games['id'] = games['id'].astype(int)

games.shape

(27220, 11)

In [6]:
games.isna().sum()

publisher       0
app_name        0
url             0
release_date    0
tags            0
reviews_url     0
specs           0
price           0
early_access    0
id              0
developer       0
dtype: int64

In [7]:
games.to_csv('data_set_limpio/games_preparado.csv',encoding='UTF-8',index=False)

In [8]:
with open('data_set_limpio/games_preparado.csv', 'rb') as f_in, gzip.open('data_set_limpio/games_preparado.csv.gz', 'wb') as f_out:
    f_out.writelines(f_in)